In [1]:
# -*- coding: utf-8 -*-
from platform import python_version
import os, time, glob, socket, pickle, joblib
import numpy as np
import pandas as pd
import scipy
from scipy.stats import linregress

print("【日期時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【工作目錄】{}".format(os.getcwd()))
print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.gethostname())))
print("【Python】{}".format(python_version()))
print("【scipy】{}".format(scipy.__version__))

【日期時間】2023/04/26 17:39:23
【工作目錄】C:\Users\88690\python\Machine Learning
【主機名稱】yiyi (192.168.68.104)
【Python】3.9.12
【scipy】1.7.3


### <font color=#0000FF>Linear Regression</font>
- OLS(Ordinary Least Squares)最小平方
    - loss function : $minimize\{SSE = \sum_{i=1}^n(y_i-\hat y_i)^2\}$
- model function
    - $y = \beta_0 + \beta_1 x_1 + \epsilon$
        - $y$ : 反應變數(又稱為應變項)
        - $x_i$ : 預測變數(又稱為自變項)
        - $\beta_0$ : 截距項(intercept)
        - $\beta_1$ : 斜率(slope)、權重(模型欲估計的參數)
        - $\hat y = \hat \beta_0 + \hat \beta_1 x_1$        
            - $\hat y、 \hat \beta_0、 \hat \beta_1$ : 被算(預測)出來的估計值(頭頂上多一頂帽子^)
    - $y = \beta_0 + \beta_1*x_1 + \beta_2*x_2 + ... + \epsilon$

In [6]:
fname1 = r"C:\Data\PyMLData\cars.csv"
fname2 = r"C:\Data\PyMLData\mtcars.csv"

cars = pd.read_csv(fname1, sep=",", engine="python", encoding="cp950")
mtcars = pd.read_csv(fname2, sep=",", engine="python", encoding="cp950")

【cars資料集】
- speed : numeric Speed (mph) 速度（mph, miles per hour 英里）
- dist : numeric Stopping distance (ft) 停止距離（ft, 英尺）

【mtcars資料集】
- mpg : Miles/(US) gallon 英里/（美國）加侖
- cyl : Number of cylinders 氣缸數
- disp : Displacement (cu.in.) 位移（cu.in.）
- hp : Gross horsepower 總馬力 
- drat : Rear axle ratio 後橋比
- wt : Weight (1000 lbs) 重量（1000 lbs）
- qsec : 1/4 mile time 1/4英里時間
- vs : Engine (0 = V-shaped, 1 = straight) 引擎 (0 = V形，1 =直)
- am : Transmission (0 = automatic, 1 = manual) 傳動 (0 =自動，1 =手動)
- gear : Number of forward gears 齒輪前進檔數
- carb : Number of carburetors 碳氫化合物數量

In [23]:
type(cars.)

pandas.core.indexes.base.Index

In [33]:
def summary_data(dataset, dataname):
    print(f"data name:{dataname}")
    print("records:{} columns:{}".format(dataset.shape[0], dataset.shape[1]))
    print("column name:{}".format(list(dataset.columns)))
    print("check na value: \n{}\n".format(dataset.isna().sum()))

In [34]:
summary_data(cars, "cars")
summary_data(mtcars, "mtcars")

data name:cars
records:50 columns:2
column name:['speed', 'dist']
check na value: 
speed    0
dist     0
dtype: int64

data name:mtcars
records:32 columns:11
column name:['mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear', 'carb']
check na value: 
mpg     0
cyl     0
disp    0
hp      0
drat    0
wt      0
qsec    0
vs      0
am      0
gear    0
carb    0
dtype: int64

